In [ ]:
# Install Libs
!pip install mediapipe
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task
!pip install mlxtend
!unzip -p "#AZAD_Lab_MH-FED_2022#" /content/gdrive/MyDrive/MH-FED_Images.zip

# Mount Drive.
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Visualisation Tools

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
def show_image(image_path):
  img = cv2.imread(image_path)
  cv2_imshow(img)

# show_image("/content/gdrive/MyDrive/images/1/Anger.jpg")

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

def get_features(image_path, show_visuals = False):
# Create an FaceLandmarker object.
  base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
  options = vision.FaceLandmarkerOptions(base_options=base_options,
                                        output_face_blendshapes=True,
                                        output_facial_transformation_matrixes=True,
                                        num_faces=1)
  detector = vision.FaceLandmarker.create_from_options(options)
  image = mp.Image.create_from_file(image_path)
  detection_result = detector.detect(image)

  if show_visuals:
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    show_image(image_path)
    cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
  return detection_result

def get_major_features(result, threshold, add_label = False, label = None):
  features_list = result.face_blendshapes[0]
  # for feature in features_list:
  #   print(feature)
  low = min(features_list, key=lambda x: x.score).score
  high = max(features_list, key=lambda x: x.score).score
  n = high - low
  # print(low, high)
  out = [feature.category_name for feature in features_list if (feature.score - low) / n >= threshold]
  if add_label:
    out.append(label)
  # for feature in features_list:
  #   print(feature.category_name, ((feature.score - low) / n), (feature.score - low) / n >= threshold)
  return out

def get_dataset_features(images_list, threshold, add_label = False, label = None):
  out = []
  for i, image in enumerate(images_list):
    print(f"{image}")
    result = get_features(image)
    out.append(get_major_features(result, threshold, add_label, label))
  return out

from collections import defaultdict, Counter
from itertools import combinations
def apriori(features, min_support=0.5, min_confidence=0.5):
  init = defaultdict(int)
  for item in features:
    for feature in item:
      init[feature] += 1
  support_value = min_support * sum(init.values())

  return init

def a2(data, min_support, min_confidence):
  rules = {}
  init = []
  for i in data:
      for q in i:
          if(q not in init):
              init.append(q)
  init = sorted(init)
  # print(init)

  c = Counter()
  for i in init:
      for d in data:
          if(i in d):
              c[i]+=1
  # print("C1:")
  # for i in c:
  #     print(str([i])+": "+str(c[i]))
  # print()
  s = int(min_support*len(init))
  l = Counter()
  for i in c:
      if(c[i] >= s):
          l[frozenset([i])]+=c[i]
  # print("L1:")
  # for i in l:
  #     print(str(list(i))+": "+str(l[i]))
  # print()
  pl = l
  pos = 1
  for count in range (2,1000):
      nc = set()
      temp = list(l)
      for i in range(0,len(temp)):
          for j in range(i+1,len(temp)):
              t = temp[i].union(temp[j])
              if(len(t) == count):
                  nc.add(temp[i].union(temp[j]))
      nc = list(nc)
      c = Counter()
      for i in nc:
          c[i] = 0
          for q in data:
              temp = set(q)
              if(i.issubset(temp)):
                  c[i]+=1
      # print("C"+str(count)+":")
      # for i in c:
      #     print(str(list(i))+": "+str(c[i]))
      # print()
      l = Counter()
      for i in c:
          if(c[i] >= s):
              l[i]+=c[i]
      # print("L"+str(count)+":")
      # for i in l:
      #     print(str(list(i))+": "+str(l[i]))
      # print()
      if(len(l) == 0):
          break
      rules.update(get_rules(get_association_rules(l, data), min_confidence))
      pl = l
      pos = count
  # print("Result: ")
  # print("L"+str(pos)+":")
  # for i in pl:
      # print(str(list(i))+": "+str(pl[i]))
  # print()
  for l in pl:
      c = [frozenset(q) for q in combinations(l,len(l)-1)]
      mmax = 0
      for a in c:
          b = l-a
          ab = l
          sab = 0
          sa = 0
          sb = 0
          for q in data:
              temp = set(q)
              if(a.issubset(temp)):
                  sa+=1
              if(b.issubset(temp)):
                  sb+=1
              if(ab.issubset(temp)):
                  sab+=1
          temp = sab/sa*100
          if(temp > mmax):
              mmax = temp
          temp = sab/sb*100
          if(temp > mmax):
              mmax = temp
          # print(str(list(a))+" -> "+str(list(b))+" = "+str(sab/sa*100)+"%")
          # print(str(list(b))+" -> "+str(list(a))+" = "+str(sab/sb*100)+"%")
      curr = 1
      # print("choosing:", end=' ')
      for a in c:
          b = l-a
          ab = l
          sab = 0
          sa = 0
          sb = 0
          for q in data:
              temp = set(q)
              if(a.issubset(temp)):
                  sa+=1
              if(b.issubset(temp)):
                  sb+=1
              if(ab.issubset(temp)):
                  sab+=1
          temp = sab/sa*100
          # if(temp == mmax):
              # print(curr, end = ' ')
          curr += 1
          temp = sab/sb*100
          # if(temp == mmax):
              # print(curr, end = ' ')
          curr += 1
      # print()
      # print()
  itemset = []
  for key in pl.keys():
    itemset.append(key)
  return itemset, rules

def get_association_rules(l_set, data):
  out = ""
  for l in l_set:
      c = [frozenset(q) for q in combinations(l,len(l)-1)]
      mmax = 0
      for a in c:
          b = l-a
          ab = l
          sab = 0
          sa = 0
          sb = 0
          for q in data:
              temp = set(q)
              if(a.issubset(temp)):
                  sa+=1
              if(b.issubset(temp)):
                  sb+=1
              if(ab.issubset(temp)):
                  sab+=1
          temp = sab/sa*100
          if(temp > mmax):
              mmax = temp
          temp = sab/sb*100
          if(temp > mmax):
              mmax = temp
          out += str(list(a))+" -> "+str(list(b))+" = "+str(sab/sa*100)+"%\n"
          out += str(list(b))+" -> "+str(list(a))+" = "+str(sab/sb*100)+"%\n"
  return out

def breakdown_rule(rule, threshold):
  rule.strip()
  if rule == "":
    return None, None
  left, right = rule.split("->")
  right, confidence = right.split("=")
  confidence = float(confidence.strip()[:-1])
  if confidence < (threshold * 100):
    return None, None
  left = left.strip()[1:-1]
  left = [x.strip()[1:-1] for x in left.split(",")]
  right = right.strip()[1:-1]
  right = [x.strip()[1:-1] for x in right.split(",")]
  return left, right
def get_rules(text, confidence):
  rules = defaultdict(set)
  rules_text = text.split("\n")
  for rule in rules_text:
    left, right = breakdown_rule(rule, confidence)
    if left is None or right is None:
      continue
    rules[frozenset(left)].update(right)
  return rules

In [ ]:
def clean_output(itemset, rules):
  for i, item in enumerate(itemset):
    itemset[i] = set(item)
  rule_list = []
  for k, v in rules.items():
    rule_list.append((set(k), set(v)))
  rule_list.sort(key = lambda x: (len(x[0]), len(x[1])))
  return itemset, rule_list

In [ ]:
FEATURE_ACCEPT_THRESHOLD = 0.7
APRIORI_SUPPORT_THRESHOLD = 0.5
APRIORI_CONFIDENCE_THRESHOLD = 1.0
labels = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprised"]

class Emotion:
  def __init__(self, itemset, rules, label):
    self.itemset = itemset
    self.rules = rules
    self.label = label

emotion_outputs = {}
for label in labels:
  features = get_dataset_features([f"/content/gdrive/MyDrive/images/{i}/{label}.jpg" for i in range(19)], FEATURE_ACCEPT_THRESHOLD, False, label)
  itemset, rules = a2(features, APRIORI_SUPPORT_THRESHOLD, APRIORI_CONFIDENCE_THRESHOLD)
  itemset, rules = clean_output(itemset, rules)
  emotion_outputs[label] = Emotion(itemset, rules, label)

for label in labels:
  print(f"Emotion - {label}")
  print("Itemsets")
  for itemset in emotion_outputs[label].itemset:
    print(itemset)
  print()
  for k, v in emotion_outputs[label].rules:
    print(f"{k} -> {v}")
  print("----------------------------------------------------------------")

In [ ]:
def extend_features(features, emotion):
  out = features
  for k, v in emotion_outputs[emotion].rules:
    if k.issubset(out):
      out.update(v)
  return out

def test_image(image_path):
  result = get_features(image_path)
  features = set(get_major_features(result, APRIORI_SUPPORT_THRESHOLD))
  out = []
  for label in labels:
    extended_features = extend_features(features, label)
    for model in emotion_outputs[label].itemset:
      if model.issubset(extended_features):
        out.append(label)
        break
  return out

print(test_image("/content/gdrive/MyDrive/s_sad.jpeg"))